In [43]:
import numpy as np
import cv2
from glob import glob

import tensorflow as tf
from tensorflow.keras import layers as L
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger

In [44]:
global image_h
global image_w
global num_landmarks


In [45]:
   """ Hyperparameters """
image_h = 256
image_w = 256
num_landmarks = 106
input_shape = (image_h, image_w, 3)
batch_size = 32
lr = 1e-3
num_epochs = 1

In [46]:
dataset_path="/kaggle/input/lp-dataset/LaPa/LaPa"
model_path="//kaggle/working/model.h5"
csv_path="/kaggle/working//data.csv"

In [47]:
def load_dataset(path):
    train_x = sorted(glob('/kaggle/input/lp-dataset/LaPa/LaPa/train/images/*.jpg'))
    train_y = sorted(glob('/kaggle/input/lp-dataset/LaPa/LaPa/train/landmarks/*.txt'))

    valid_x = sorted(glob('/kaggle/input/lp-dataset/LaPa/LaPa/val/images/*.jpg'))
    valid_y = sorted(glob('/kaggle/input/lp-dataset/LaPa/LaPa/val/landmarks/*.txt'))

    test_x = sorted(glob('/kaggle/input/lp-dataset/LaPa/LaPa/test/images/*.jpg'))
    test_y = sorted(glob('/kaggle/input/lp-dataset/LaPa/LaPa/test/landmarks/*.txt'))  
    
    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

    

In [48]:
def read_image_lankmarks(image_path, landmark_path):
    """ Image """
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    h, w, _ = image.shape
    image = cv2.resize(image, (256, 256))
    image = image/255.0
    image = image.astype(np.float32)
    data = open(landmark_path, "r").read()
    lankmarks = []
    
    for line in data.strip().split("\n")[1:]:
        x, y = line.split(" ")
        x = float(x)/w
        y = float(y)/h  
        
        lankmarks.append(x)
        lankmarks.append(y)
    
    lankmarks = np.array(lankmarks, dtype=np.float32)
        
    return image, lankmarks   

In [49]:
def preprocess(x, y):
    def f(x, y):
        x = x.decode()
        y = y.decode()

        image, landmarks = read_image_lankmarks(x, y)
        return image, landmarks

    image, landmarks = tf.numpy_function(f, [x, y], [tf.float32, tf.float32])
    image.set_shape([256,256, 3])
    landmarks.set_shape([106 * 2])

    return image, landmarks

In [50]:
def tf_dataset(x, y, batch=8):
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    ds = ds.shuffle(buffer_size=5000).map(preprocess)
    ds = ds.batch(batch).prefetch(2)
    return ds

In [51]:
def build_model(input_shape, num_landmarks):
    inputs = L.Input(input_shape)

    backbone = MobileNetV2(include_top=False, weights="imagenet", input_tensor=inputs, alpha=0.5)
    backbone.trainable = True

    x = backbone.output
    x = L.GlobalAveragePooling2D()(x)
    x = L.Dropout(0.2)(x)
    outputs = L.Dense(num_landmarks*2, activation="sigmoid")(x)

    model = tf.keras.models.Model(inputs, outputs)
    return model

In [52]:
if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
   
    """ Hyperparameters """
   

    """ Paths """
  

    """ Loading the dataset """
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)
    print(f"Train: {len(train_x)}/{len(train_y)} - Valid: {len(valid_x)}/{len(valid_y)} - Test: {len(test_x)}/{len(test_x)}")
    print("")

    """ Dataset Pipeline """
    train_ds = tf_dataset(train_x, train_y, batch=batch_size)
    valid_ds = tf_dataset(valid_x, valid_y, batch=batch_size)

    """ Model """
    model = build_model(input_shape, num_landmarks)
    model.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(lr),metrics=["mae","accuracy"])

    """ Training """
    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True, monitor='val_loss'),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path, append=True),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
    ]

    model.fit(train_ds,
        validation_data=valid_ds,
        epochs=num_epochs,
        callbacks=callbacks
    )

Train: 18168/18168 - Valid: 2000/2000 - Test: 2000/2000

568/568 [==============================] - 242s 418ms/step - loss: 0.6564 - mae: 0.0235 - accuracy: 0.3212 - val_loss: 1.6217 - val_mae: 0.2973 - val_accuracy: 0.4070

Epoch 00001: val_loss improved from inf to 1.62170, saving model to //kaggle/working/model.h5


In [53]:
model.save("model.h5")